### 교차 검증 실습

In [2]:
import os, warnings
import numpy as np
# 경고 메세지 무시하거나 숨길때(ignore), 다시보이게(default)
# warnings.filterwarnings(action='ignore')
warnings.filterwarnings(action='ignore')

In [3]:
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd

In [4]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
train.shape, test.shape

((891, 12), (418, 11))

### 데이터 확인

In [46]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [47]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


### 결측치 처리

In [48]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [49]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


#### Age 결측치 평균으로 채우기

In [50]:
train.Age = train.Age.fillna(train.Age.mean())
test.Age = test.Age.fillna(train.Age.mean())
train.info(), test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          891 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass  

(None, None)

In [51]:
train.Embarked.value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [52]:
test.Embarked.value_counts()

S    270
C    102
Q     46
Name: Embarked, dtype: int64

In [53]:
train.Embarked = train.Embarked.fillna('S')
test.Embarked = test.Embarked.fillna('S')
train.info(), test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          891 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     891 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass  

(None, None)

### 데이터 형태 수치형으로 변환

In [54]:
train.Sex.value_counts()

male      577
female    314
Name: Sex, dtype: int64

In [55]:
test.Sex.value_counts()

male      266
female    152
Name: Sex, dtype: int64

In [56]:
train['Sex_n'] = train.Sex.map({'male':1, 'female':2})
test['Sex_n'] = test.Sex.map({'male':1, 'female':2})
train.info(), test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          891 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     891 non-null    object 
 12  Sex_n        891 non-null    int64  
dtypes: float64(2), int64(6), object(5)
memory usage: 90.6+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Passenge

(None, None)

In [57]:
train['Embarked_n'] = train['Embarked'].map({'S':1, 'C':2, 'Q':3})
test['Embarked_n'] = test['Embarked'].map({'S':1, 'C':2, 'Q':3})
train.info(), test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          891 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     891 non-null    object 
 12  Sex_n        891 non-null    int64  
 13  Embarked_n   891 non-null    int64  
dtypes: float64(2), int64(7), object(5)
memory usage: 97.6+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------  

(None, None)

### 데이터 분할 및 특성 선택

In [58]:
sel = ['Pclass', 'Age', 'Sex_n', 'Embarked_n']
X = train[sel]
y = train['Survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((668, 4), (668,), (223, 4), (223,))

### 모델 생성, 교차 검증

In [59]:
knn = KNeighborsClassifier()
knn.fit(X_train,  y_train)
# kfold = KFold(n_splits=cv 개수, shuffle=샘플을 섞는 여부)
kfold = KFold(n_splits=5, shuffle=True)
# scores = cross_val_score(모델 객체, 입력, 결과, cv = cross_validation)
score = cross_val_score(knn, X_test, y_test, cv=kfold)
print(f"교차 검증 점수 : {score}")
print("교자 검증 점수 평균 :", score.mean())

교차 검증 점수 : [0.6        0.73333333 0.62222222 0.70454545 0.68181818]
교자 검증 점수 평균 : 0.6683838383838383


### knn 모델 test_size 조절하여 정확도 올리기

##### 훈련 : 테스트 = 9 : 1

In [60]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state=0)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((801, 4), (801,), (90, 4), (90,))

In [61]:
knn.fit(X_train,  y_train)
# kfold = KFold(n_splits=cv 개수, shuffle=샘플을 섞는 여부)
kfold = KFold(n_splits=5, shuffle=True)
# scores = cross_val_score(모델 객체, 입력, 결과, cv = cross_validation)
score = cross_val_score(knn, X_test, y_test, cv=kfold)
print(f"교차 검증 점수 : {score}")
print("교자 검증 점수 평균 :", score.mean())

교차 검증 점수 : [0.61111111 0.72222222 0.66666667 0.61111111 0.5       ]
교자 검증 점수 평균 : 0.6222222222222222


##### 훈련 : 테스트 = 8 : 2

In [62]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=0)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((712, 4), (712,), (179, 4), (179,))

In [63]:
knn.fit(X_train,  y_train)
# kfold = KFold(n_splits=cv 개수, shuffle=샘플을 섞는 여부)
kfold = KFold(n_splits=5, shuffle=True)
# scores = cross_val_score(모델 객체, 입력, 결과, cv = cross_validation)
score = cross_val_score(knn, X_test, y_test, cv=kfold)
print(f"교차 검증 점수 : {score}")
print("교자 검증 점수 평균 :", score.mean())

교차 검증 점수 : [0.69444444 0.72222222 0.66666667 0.55555556 0.62857143]
교자 검증 점수 평균 : 0.6534920634920633


##### 훈련 : 테스트 = 7 : 3

In [64]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=0)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((623, 4), (623,), (268, 4), (268,))

In [65]:
knn.fit(X_train,  y_train)
# kfold = KFold(n_splits=cv 개수, shuffle=샘플을 섞는 여부)
kfold = KFold(n_splits=5, shuffle=True)
# scores = cross_val_score(모델 객체, 입력, 결과, cv = cross_validation)
score = cross_val_score(knn, X_test, y_test, cv=kfold)
print(f"교차 검증 점수 : {score}")
print("교자 검증 점수 평균 :", score.mean())

교차 검증 점수 : [0.68518519 0.61111111 0.66666667 0.73584906 0.73584906]
교자 검증 점수 평균 : 0.686932215234102


##### 훈련 : 테스트 = 6 : 4

In [66]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4, random_state=0)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((534, 4), (534,), (357, 4), (357,))

In [67]:
knn.fit(X_train,  y_train)
# kfold = KFold(n_splits=cv 개수, shuffle=샘플을 섞는 여부)
kfold = KFold(n_splits=5, shuffle=True)
# scores = cross_val_score(모델 객체, 입력, 결과, cv = cross_validation)
score = cross_val_score(knn, X_test, y_test, cv=kfold)
print(f"교차 검증 점수 : {score}")
print("교자 검증 점수 평균 :", score.mean())

교차 검증 점수 : [0.70833333 0.73611111 0.74647887 0.71830986 0.70422535]
교자 검증 점수 평균 : 0.7226917057902973


##### 훈련 : 테스트 = 5 : 5

In [68]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.5, random_state=0)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((445, 4), (445,), (446, 4), (446,))

In [69]:
knn.fit(X_train,  y_train)
# kfold = KFold(n_splits=cv 개수, shuffle=샘플을 섞는 여부)
kfold = KFold(n_splits=5, shuffle=True)
# scores = cross_val_score(모델 객체, 입력, 결과, cv = cross_validation)
score = cross_val_score(knn, X_test, y_test, cv=kfold)
print(f"교차 검증 점수 : {score}")
print("교자 검증 점수 평균 :", score.mean())

교차 검증 점수 : [0.76666667 0.73033708 0.70786517 0.74157303 0.70786517]
교자 검증 점수 평균 : 0.7308614232209738


#### test_size 조절해서 교차 검증 점수 출력

In [70]:
for i in range(1, 6):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = (i/10), random_state=0)
    knn.fit(X_train,  y_train)
    # kfold = KFold(n_splits=cv 개수, shuffle=샘플을 섞는 여부)
    kfold = KFold(n_splits=5, shuffle=True)
    # scores = cross_val_score(모델 객체, 입력, 결과, cv = cross_validation )
    score = cross_val_score(knn, X_test, y_test, cv=kfold)
    print(f"훈련 : 테스트 = {10-i} : {i}")
    print(f"교차 검증 점수 : {score}")
    print("교자 검증 점수 평균 :", score.mean())
    print()

훈련 : 테스트 = 9 : 1
교차 검증 점수 : [0.61111111 0.66666667 0.61111111 0.55555556 0.5       ]
교자 검증 점수 평균 : 0.5888888888888889

훈련 : 테스트 = 8 : 2
교차 검증 점수 : [0.61111111 0.69444444 0.63888889 0.58333333 0.68571429]
교자 검증 점수 평균 : 0.6426984126984128

훈련 : 테스트 = 7 : 3
교차 검증 점수 : [0.77777778 0.64814815 0.77777778 0.67924528 0.64150943]
교자 검증 점수 평균 : 0.7048916841369671

훈련 : 테스트 = 6 : 4
교차 검증 점수 : [0.625      0.70833333 0.66197183 0.70422535 0.73239437]
교자 검증 점수 평균 : 0.6863849765258216

훈련 : 테스트 = 5 : 5
교차 검증 점수 : [0.78888889 0.70786517 0.79775281 0.68539326 0.7752809 ]
교자 검증 점수 평균 : 0.75103620474407



## 데이터의 'Age'를 median 값으로 채워서 검증 하기

In [71]:
# 데이터 읽기
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
# 결측값 채우기(Age)
train.Age = train.Age.fillna(train.Age.median())
test.Age = test.Age.fillna(train.Age.median())
# 결측값 채우기(Embarked)
train.Embarked = train.Embarked.fillna('S')
test.Embarked = test.Embarked.fillna('S')
# 데이터 수치형으로 변환(Sex)
train['Sex_n'] = train.Sex.map({'male':1, 'female':2})
test['Sex_n'] = test.Sex.map({'male':1, 'female':2})
# 데이터 수치형으로 변환(Embarked)
train['Embarked_n'] = train['Embarked'].map({'S':1, 'C':2, 'Q':3})
test['Embarked_n'] = test['Embarked'].map({'S':1, 'C':2, 'Q':3})
# 데이터 특성 선택 및 분할
sel = ['Pclass', 'Age', 'Sex_n', 'Embarked_n']
X = train[sel]
y = train['Survived']
# 모델 생성
knn = KNeighborsClassifier()
# test_size 변경
for i in range(1, 6):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = (i/10), random_state=0)
    # 모델 훈련
    knn.fit(X_train,  y_train)
    # kfold = KFold(n_splits=cv 개수, shuffle=샘플을 섞는 여부)
    kfold = KFold(n_splits=5, shuffle=True)
    # scores = cross_val_score(모델 객체, 입력, 결과, cv = cross_validation)
    score = cross_val_score(knn, X_test, y_test, cv=kfold)
    print(f"훈련 : 테스트 = {10-i} : {i}")
    print(f"교차 검증 점수 : {score}")
    print("교자 검증 점수 평균 :", score.mean())
    print()

훈련 : 테스트 = 9 : 1
교차 검증 점수 : [0.5        0.38888889 0.55555556 0.66666667 0.61111111]
교자 검증 점수 평균 : 0.5444444444444445

훈련 : 테스트 = 8 : 2
교차 검증 점수 : [0.69444444 0.69444444 0.61111111 0.86111111 0.51428571]
교자 검증 점수 평균 : 0.6750793650793651

훈련 : 테스트 = 7 : 3
교차 검증 점수 : [0.72222222 0.66666667 0.64814815 0.66037736 0.71698113]
교자 검증 점수 평균 : 0.6828791055206149

훈련 : 테스트 = 6 : 4
교차 검증 점수 : [0.73611111 0.65277778 0.76056338 0.73239437 0.64788732]
교자 검증 점수 평균 : 0.7059467918622848

훈련 : 테스트 = 5 : 5
교차 검증 점수 : [0.74444444 0.76404494 0.68539326 0.83146067 0.82022472]
교자 검증 점수 평균 : 0.7691136079900125



## 데이터 분할 조정
* 'Age' : median
* statify 설정

In [72]:
# 데이터 읽기
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
# 결측값 채우기(Age)
train.Age = train.Age.fillna(train.Age.median())
test.Age = test.Age.fillna(train.Age.median())
# 결측값 채우기(Embarked)
train.Embarked = train.Embarked.fillna('S')
test.Embarked = test.Embarked.fillna('S')
# 데이터 수치형으로 변환(Sex)
train['Sex_n'] = train.Sex.map({'male':1, 'female':2})
test['Sex_n'] = test.Sex.map({'male':1, 'female':2})
# 데이터 수치형으로 변환(Embarked)
train['Embarked_n'] = train['Embarked'].map({'S':1, 'C':2, 'Q':3})
test['Embarked_n'] = test['Embarked'].map({'S':1, 'C':2, 'Q':3})
# 데이터 특성 선택 및 분할
sel = ['Pclass', 'Age', 'Sex_n', 'Embarked_n']
X = train[sel]
y = train['Survived']
# 모델 생성
knn = KNeighborsClassifier()
# test_size 변경
for i in range(1, 6):
    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size = (i/10), random_state=0)
    # 모델 훈련
    knn.fit(X_train,  y_train)
    # kfold = KFold(n_splits=cv 개수, shuffle=샘플을 섞는 여부)
    kfold = KFold(n_splits=5, shuffle=True)
    # scores = cross_val_score(모델 객체, 입력, 결과, cv = cross_validation)
    score = cross_val_score(knn, X_test, y_test, cv=kfold)
    print(f"훈련 : 테스트 = {10-i} : {i}")
    print(f"교차 검증 점수 : {score}")
    print("교자 검증 점수 평균 :", score.mean())
    print()

훈련 : 테스트 = 9 : 1
교차 검증 점수 : [0.72222222 0.72222222 0.72222222 0.5        0.44444444]
교자 검증 점수 평균 : 0.6222222222222221

훈련 : 테스트 = 8 : 2
교차 검증 점수 : [0.66666667 0.69444444 0.63888889 0.58333333 0.74285714]
교자 검증 점수 평균 : 0.6652380952380953

훈련 : 테스트 = 7 : 3
교차 검증 점수 : [0.66666667 0.7037037  0.72222222 0.69811321 0.77358491]
교자 검증 점수 평균 : 0.712858141160028

훈련 : 테스트 = 6 : 4
교차 검증 점수 : [0.76388889 0.81944444 0.78873239 0.74647887 0.81690141]
교자 검증 점수 평균 : 0.7870892018779342

훈련 : 테스트 = 5 : 5
교차 검증 점수 : [0.74444444 0.76404494 0.7752809  0.70786517 0.83146067]
교자 검증 점수 평균 : 0.7646192259675406



## RandomForest 모델
* Age : median
* statify 설정

In [5]:
from sklearn.ensemble import RandomForestClassifier

In [7]:
# 데이터 읽기
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
# 결측값 채우기(Age)
train.Age = train.Age.fillna(train.Age.median())
test.Age = test.Age.fillna(train.Age.median())
# 결측값 채우기(Embarked)
train.Embarked = train.Embarked.fillna('S')
test.Embarked = test.Embarked.fillna('S')
# 데이터 수치형으로 변환(Sex)
train['Sex_n'] = train.Sex.map({'male':1, 'female':2})
test['Sex_n'] = test.Sex.map({'male':1, 'female':2})
# 데이터 수치형으로 변환(Embarked)
train['Embarked_n'] = train['Embarked'].map({'S':1, 'C':2, 'Q':3})
test['Embarked_n'] = test['Embarked'].map({'S':1, 'C':2, 'Q':3})
# 데이터 특성 선택 및 분할
sel = ['Pclass', 'Age', 'Sex_n', 'Embarked_n']
X = train[sel]
y = train['Survived']
# 모델 생성
rf_c = RandomForestClassifier()
# test_size 변경
for i in range(1, 6):
    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size = (i/10), random_state=0)
    # 모델 훈련
    rf_c.fit(X_train,  y_train)
    # kfold = KFold(n_splits=cv 개수, shuffle=샘플을 섞는 여부)
    kfold = KFold(n_splits=5, shuffle=True)
    # scores = cross_val_score(모델 객체, 입력, 결과, cv = cross_validation)
    score = cross_val_score(rf_c, X_test, y_test, cv=kfold)
    print(f"훈련 : 테스트 = {10-i} : {i}")
    print(f"교차 검증 점수 : {score}")
    print("교자 검증 점수 평균 :", score.mean())
    print()

훈련 : 테스트 = 9 : 1
교차 검증 점수 : [0.61111111 0.55555556 0.77777778 0.61111111 0.83333333]
교자 검증 점수 평균 : 0.6777777777777778

훈련 : 테스트 = 8 : 2
교차 검증 점수 : [0.83333333 0.77777778 0.80555556 0.77777778 0.62857143]
교자 검증 점수 평균 : 0.7646031746031746

훈련 : 테스트 = 7 : 3
교차 검증 점수 : [0.75925926 0.77777778 0.83333333 0.90566038 0.83018868]
교자 검증 점수 평균 : 0.8212438853948288

훈련 : 테스트 = 6 : 4
교차 검증 점수 : [0.79166667 0.80555556 0.78873239 0.8028169  0.74647887]
교자 검증 점수 평균 : 0.7870500782472614

훈련 : 테스트 = 5 : 5
교차 검증 점수 : [0.73333333 0.78651685 0.78651685 0.83146067 0.83146067]
교자 검증 점수 평균 : 0.7938576779026217

